# Build Dataset
This is a notebook that asks questions and based on those reposnes builds the dataset. The questions are loaded from Google Gemini

In [5]:
import os
import google.generativeai as genai
import pandas as pd
import datetime
from dotenv import load_dotenv
import numpy as np

load_dotenv()

def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False


genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": .5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]
now = datetime.datetime.now()
model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  safety_settings=safety_settings,
  generation_config=generation_config,
  system_instruction="use this number as a random seed:"+now.strftime("%m-%d-%H-%M")+ ". Output a random scenario followed by an action by a player. Display it in this format: <DM>You encounter a heavily encrypted device with multiple security layers. </DM>\n<Player>I attempt to bypass the encryption using my understanding of systems and logic.</Player>",
)

chat_session = model.start_chat(
    history=[]
)

response_dict = {"description":[], "chance-success":[]}
user_input = None

while user_input!="quit":

    response = chat_session.send_message(now.strftime("%Y-%m-%d %H:%M:%S"))
    model_response = response.text
    attributes = np.random.rand(7)
    output = f'health:{attributes[0]:.2f}\nstrength:{attributes[1]:.2f}\ndexterity:{attributes[2]:.2f}\nperception:{attributes[3]:.2f}\nintelligence:{attributes[4]:.2f}\ncharisma:{attributes[5]:.2f}\nstamina:{attributes[6]:.2f}\n{model_response}'
    print(output)

    user_input = input()
    
    if is_float(user_input):
        response_dict["description"].append(output)
        response_dict["chance-success"].append(float(user_input))
    else:
        print("Invalid float")
    
print(response_dict)

df = pd.DataFrame.from_dict(response_dict)
now = datetime.datetime.now()
df.to_csv("../data/risk_dataset_"+now.strftime("%m-%d-%H-%M")+".csv", index=False)

health:0.06
strength:0.72
dexterity:0.14
perception:0.44
intelligence:0.04
charisma:0.44
stamina:0.15
<DM>You are tracking a smuggling ring through the bustling marketplace of a desert port city.  You've followed them to a dimly lit warehouse, the air thick with the smell of exotic spices and something vaguely metallic.  Guards patrol the perimeter, their eyes constantly scanning the crowds. </DM>
<Player>I disguise myself as a merchant selling rugs and attempt to blend in with the market activity, moving closer to the warehouse entrance to eavesdrop.</Player>

health:0.42
strength:0.21
dexterity:0.10
perception:0.14
intelligence:0.19
charisma:0.04
stamina:0.08
<DM>You are trekking through a dense jungle, the humidity clinging to you like a second skin.  You hear a rustling in the undergrowth ahead, followed by a low growl.  The leaves part to reveal a jaguar, its eyes fixed on you with predatory intent. </DM>
<Player>I slowly back away, keeping my eyes locked on the jaguar while reach